In [2]:
from tqdm import tqdm
from pathlib import Path
import cv2
import datetime
import boto3
from botocore.client import Config

In [3]:
s3 = boto3.resource(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="username",
    aws_secret_access_key="password",
    config=Config(signature_version="s3v4"),
    region_name="us-east-1",
)

In [4]:
with open("./batch_0.txt") as f:
    paths = [
        list(map(lambda x: x.strip(), string.split("\t"))) for string in f.readlines()
    ]

In [5]:
DATA_DIR = Path("data")

In [7]:
project = "online_fetching"

for path in tqdm(paths):
    path, label = path
    img = cv2.imread(str(DATA_DIR / Path(path).name))
    if img is None:
        continue

    height, width = img.shape[:2]

    s3.Bucket("data").upload_file(
        str(DATA_DIR / Path(path).name),
        f"{project}/{ datetime.datetime.now().strftime('%Y%m%d')}/{Path(path).name}",
        ExtraArgs={
            "ContentType": "image/jpeg",
            "Metadata": {"height": str(height), "width": str(width)},
        },
    )

100%|██████████| 50000/50000 [18:58<00:00, 43.94it/s]
